<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">


# PROJECT-2. Анализ вакансий из HeadHunter

In [2]:
import psycopg2
import pandas as pd
import urllib.parse as req

### выполняем соединение с базами данных 

In [3]:
connection = psycopg2.connect(
   dbname=DBNAME,
   user=USER,
   host=HOST,
   password=PASSWORD,
   port=PORT
)

## Предварительный анализ данных
#### проводим разведовательный анализ наших БД по основным общим параметрам
#### Узнаем общее количество вакансий 

In [4]:

query3_1 = f'''select count(id) as "Общее число вакансий"
                from VACANCIES 
            '''
df3_1 = pd.read_sql_query(query3_1,connection)
df3_1

C:\Users\Lermars asus nb\AppData\Local\Temp\ipykernel_11176\4089202730.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df3_1 = pd.read_sql_query(query3_1,connection)


,Общее число вакансий
0,49197


#### узнаем общее количество уникалных работодателей

In [5]:

query3_2 = f'''select count(id) as "Количество работодателей"
               FROM employers 
            '''
df3_2 = pd.read_sql_query(query3_2,connection)
df3_2

C:\Users\Lermars asus nb\AppData\Local\Temp\ipykernel_11176\2180137041.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df3_2 = pd.read_sql_query(query3_2,connection)


,Количество работодателей
0,23501


#### Узнаем количество регионов с из БД с регионами . Которое нам пригодится для поиска вакансий или мест регистрации работодателей

In [6]:

query3_3 = f'''select count(id) as "Количество регионов"
               FROM areas
            '''
df3_3 = pd.read_sql_query(query3_3,connection)
df3_3

C:\Users\Lermars asus nb\AppData\Local\Temp\ipykernel_11176\1623985449.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df3_3 = pd.read_sql_query(query3_3,connection)


,Количество регионов
0,1362


#### Узнаем общее количество сфер деятельности которое присутствует в наших иследованиях

In [7]:
query3_4 = f'''select count(id) as "Количество сфер деятельности"
               FROM INDUSTRIES 
             '''
df3_4 = pd.read_sql_query(query3_4,connection)
df3_4

C:\Users\Lermars asus nb\AppData\Local\Temp\ipykernel_11176\3633457808.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df3_4 = pd.read_sql_query(query3_4,connection)


,Количество сфер деятельности
0,294


## Детальный анализ вакансий

#### Узнаем топ-5 городов по числу вакансий

In [8]:
query4_1 = f'''select VACANCIES.area_id,
            COUNT(DISTINCT VACANCIES.id) as cnt,
            areas.name
            from VACANCIES 
            LEFT JOIN areas ON VACANCIES.area_id = areas.id
            GROUP BY VACANCIES.area_id, areas.name
           ORDER BY cnt desc
        '''
df4_1 = pd.read_sql_query(query4_1, connection)
df4_1.head(5)

C:\Users\Lermars asus nb\AppData\Local\Temp\ipykernel_11176\4088171825.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df4_1 = pd.read_sql_query(query4_1, connection)


,area_id,cnt,name
0,1,5333,Москва
1,2,2851,Санкт-Петербург
2,1002,2112,Минск
3,4,2006,Новосибирск
4,160,1892,Алматы


#### Узнаем число вакансий в которых указанна хотябы одна из границ (нижняя или верхняя) ЗП

In [9]:

query4_2 = f'''select count(id)  as "Вакансии с указаными ЗП"
               FROM VACANCIES 
               WHERE (salary_from > 1) or (salary_to > 1)
            '''
df4_2 = pd.read_sql_query(query4_2, connection)
df4_2

C:\Users\Lermars asus nb\AppData\Local\Temp\ipykernel_11176\1823314710.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df4_2 = pd.read_sql_query(query4_2, connection)


,Вакансии с указаными ЗП
0,24073


#### Узнаем среднию заработную плату . по нижней и верхней границе . В тех резюме в которых она указана . 

In [10]:
query4_3 = f'''select AVG(salary_from)  as "Средняя значение нижней граници ЗП",
                      AVG(salary_to) as "Средняя значение верхней граници ЗП"
               FROM VACANCIES 
            '''
df4_3 = pd.read_sql_query(query4_3, connection)
df4_3

C:\Users\Lermars asus nb\AppData\Local\Temp\ipykernel_11176\2877578300.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df4_3 = pd.read_sql_query(query4_3, connection)


,Средняя значение нижней граници З,Средняя значение верхней граници
0,71064.657901,110536.741923


#### Узнаем количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. 

In [11]:
query4_4 = f'''SELECT schedule, employment, COUNT(*) AS vacancy_count
            FROM VACANCIES
            GROUP BY schedule, employment
            ORDER BY vacancy_count DESC
        '''
df4_4 = pd.read_sql_query(query4_4, connection)
df4_4

C:\Users\Lermars asus nb\AppData\Local\Temp\ipykernel_11176\4240994516.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df4_4 = pd.read_sql_query(query4_4, connection)


,schedule,employment,vacancy_count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


#### Отсюда мы видем что второй по популярности свяязкой является "Удаленная работа - Полная занятость"

#### Узнаем количество вакансий в зависимости от требуемого опыта 

In [12]:
query4_5  = f'''SELECT experience, count(experience) as exp
                from VACANCIES
                GROUP BY experience
                order by exp ASC
             '''
df4_5 = pd.read_sql_query(query4_5, connection)
df4_5

C:\Users\Lermars asus nb\AppData\Local\Temp\ipykernel_11176\555336061.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df4_5 = pd.read_sql_query(query4_5, connection)


,experience,exp
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


#### Выесняем какие работодатели находятся на 1 и 5 месте по числу вакансий 

## Анализ работодателей

In [13]:
query5_1  = f'''SELECT EMPLOYERS.name, COUNT(VACANCIES.employer_id) as com_name
                from VACANCIES
                LEFT join EMPLOYERS ON EMPLOYERS.id = VACANCIES.employer_id
                GROUP BY EMPLOYERS.name
                order by com_name DESC
                    LIMIT 6
            '''
df5_1 = pd.read_sql_query(query5_1, connection)
print(df5_1.loc[[0]])
print(df5_1.loc[[4]])

C:\Users\Lermars asus nb\AppData\Local\Temp\ipykernel_11176\2394077185.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df5_1 = pd.read_sql_query(query5_1, connection)


     name  com_name
0  Яндекс      1933
            name  com_name
4  Газпром нефть       331


#### Таким образом мы выяснили что лидер по вакансиям Яндекс . А пятое место занимает Газпром нефть

#### Узнаем регион в котором наибольшее количество работодателей и отсутствуют вакансии.

In [14]:
query5_2  = f'''SELECT a.name,
                COUNT(DISTINCT e.id) AS employer_count, COUNT(v.area_id) AS vacancy_count
                FROM areas a
                LEFT JOIN employers e ON a.id = e.area
                LEFT JOIN vacancies v ON e.id = v.employer_id AND v.area_id = a.id
                GROUP BY a.name
            '''
df5_2 = pd.read_sql_query(query5_2, connection)
df5_2.sort_values (by = ['employer_count', 'vacancy_count'], ascending = [ False , True ])
filtered_df = df5_2[df5_2['vacancy_count'] == 0]
result = filtered_df[filtered_df['employer_count'] == filtered_df['employer_count'].max()]['name'].iloc[0]
print(result)

C:\Users\Lermars asus nb\AppData\Local\Temp\ipykernel_11176\3166509902.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df5_2 = pd.read_sql_query(query5_2, connection)


Россия


#### От сюда мы вяснили что наибольшее Количество работодателей зарегистрированно в регионне "России". И логично что нету вакансий в данном регионе тк это страна. А вакансии размещаюся с указанием города внутри страны . 

#### Узнаем количество регионов в котором работадатель разместил предложения с вакансиями

In [15]:
query5_3 = f'''SELECT e.id as ID_Company,
            e.name,
            COUNT(DISTINCT v.area_id) as sum
            FROM EMPLOYERS e
            LEFT JOIN VACANCIES v ON v.employer_id = e.id
            GROUP by ID_Company
            ORDER BY sum DESC
        '''
df5_3 = pd.read_sql_query(query5_3, connection)
df5_3

C:\Users\Lermars asus nb\AppData\Local\Temp\ipykernel_11176\1991937316.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df5_3 = pd.read_sql_query(query5_3, connection)


,id_company,name,sum
0,1740,Яндекс,181
1,2748,Ростелеком,152
2,5724811,Спецремонт,116
3,5130287,Поляков Денис Иванович,88
4,3682876,ООО ЕФИН,71
...,...,...,...
23496,5287515,Но Смокинг,0
23497,2839354,МАСТЕРСКАЯ CREED,0
23498,5774578,Dzing Finance,0
23499,5775069,КОРС,0


#### больше всего регионов с  вакансиями у компании Яндекс (181 регион)

#### Узнаем  количества работодателей, у которых не указана сфера деятельности.

In [16]:
query5_4 = f'''SELECT count(*) as "без сферы деятельности"
              FROM EMPLOYERS_INDUSTRIES e
              FULL OUTER JOIN EMPLOYERS i ON e.EMPLOYER_id = i.id
              WHERE e.EMPLOYER_id is NULL
           '''
df5_4 = pd.read_sql_query(query5_4, connection)
df5_4

C:\Users\Lermars asus nb\AppData\Local\Temp\ipykernel_11176\3923737123.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df5_4 = pd.read_sql_query(query5_4, connection)


,без сферы деятельности
0,8419


#### Узнаем название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности.

In [17]:
query5_5 = f'''SELECT e.name
              From EMPLOYERS_INDUSTRIES i
              Left join EMPLOYERS e ON i.employer_id = e.id
              GROUP BY e.name 
              HAVING count(distinct i.industry_id) = 4
           '''
df5_5 = pd.read_sql_query(query5_5, connection)
print(df5_5.iloc[2])

name    2ГИС
Name: 2, dtype: object


C:\Users\Lermars asus nb\AppData\Local\Temp\ipykernel_11176\3364034220.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df5_5 = pd.read_sql_query(query5_5, connection)


#### Узнаем у какого количества работодателей в качестве сферы деятельности указана «Разработка программного обеспечения».

In [18]:
query5_6 = f'''SELECT count(DISTINCT e.employer_id) as sum
              FROM EMPLOYERS_INDUSTRIES e
              JOIN INDUSTRIES i on e.industry_id = i.id
              WHere i.name = 'Разработка программного обеспечения'
           '''
df5_6 = pd.read_sql_query(query5_6, connection)
df5_6

C:\Users\Lermars asus nb\AppData\Local\Temp\ipykernel_11176\2753498998.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df5_6 = pd.read_sql_query(query5_6, connection)


,sum
0,3553


#### Для компании «Яндекс» выведим список регионов-миллионников , в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавляем строку Total с общим количеством вакансий компании, собранных в этой таблице. Список городов мы берем с википедии

In [19]:
path_common = req.urlparse('https://ru.wikipedia.org/wiki/Города-миллионеры_России#Список_городов-миллионеров')
url =path_common.scheme + '://' + req.quote(path_common.netloc) + req.quote(path_common.path) + '#' +req.quote(path_common.fragment)
cities = pd.read_html(url)[1]
cities_r = tuple(cities['Город'].tolist())
query5_7 = f'''SELECT *
              from VACANCIES v
              JOIN AREAS a ON v.area_id = a.id
              WHERE employer_id = 1740
              GROUP BY v.id, a.id
            '''
df5_7 = pd.read_sql_query(query5_7, connection)
filtered_df = df5_7[df5_7['name'].isin(cities_r)]
result = filtered_df.groupby('name').size().reset_index(name='cnt')
total = result['cnt'].sum()
display(result)
display(total)


C:\Users\Lermars asus nb\AppData\Local\Temp\ipykernel_11176\2301531068.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df5_7 = pd.read_sql_query(query5_7, connection)


,name,cnt
0,Волгоград,24
1,Воронеж,32
2,Екатеринбург,39
3,Казань,25
4,Краснодар,30
5,Красноярск,23
6,Москва,54
7,Нижний Новгород,36
8,Новосибирск,35
9,Омск,21


485

#### Мы получили список из 16 городов с количеством вакинсий от компании Яндекс 
#### И общуюю сумму вакансий по этим городам (485)

## Предметный анализ

#### Узнаем сколько вакансий имеет отношении к данным 
##### мы исходим с того что в названии вакансий дожно быть 'данн' 'data'

In [20]:
query6_1 = f'''SELECT COUNT(*)
               From vacancies 
               Where name ILIKE '%данн%'  or name ILIKE '%data%'
            '''
df6_1 = pd.read_sql_query(query6_1, connection)
df6_1

C:\Users\Lermars asus nb\AppData\Local\Temp\ipykernel_11176\1126451511.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df6_1 = pd.read_sql_query(query6_1, connection)


,count
0,1771


#### Узнаем сколько есть подходящих вакансий для начинающего дата-сайентиста?


In [21]:
query6_2 = f'''
            SELECT count(*)
            FROM vacancies        
            WHERE (
                (LOWER(name) LIKE '%data scientist%'
                OR LOWER(name) LIKE '%data science%'
                OR LOWER(name) LIKE '%исследователь данных%'
                OR LOWER(name) LIKE '%ml%'
                OR LOWER(name) LIKE '%machine learning%'
                OR LOWER(name) LIKE '%машинн%обучен%')
                AND LOWER(name) NOT LIKE '%html%'
            )
            
            AND (
                LOWER(name) LIKE '%junior%'
                OR LOWER(experience) LIKE '%нет опыта%'
                OR LOWER(employment) LIKE '%стажировка%')
            '''
df6_2 = pd.read_sql_query(query6_2,connection)
df6_2

C:\Users\Lermars asus nb\AppData\Local\Temp\ipykernel_11176\3023095685.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df6_2 = pd.read_sql_query(query6_2,connection)


,count
0,51


#### узнаем сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

In [22]:
query6_3= f'''select COUNT(*)
              from vacancies
              where
                    (
                    lower(name) like '%data scientist%'  or
                    lower(name) like '%data science%' or
                    lower(name) like '%исследователь данных%'  or
                    (name like '%ML%' and name not ilike '%HTML%')  or
                    lower(name) like '%machine learning%'  or
                    lower(name) like '%машинн%обучен%'
                    )
                    and
                    (
                    key_skills ilike '%sql%' or key_skills ilike '%postres%' 
                )
           '''
df6_3 = pd.read_sql_query(query6_3, connection)
df6_3

C:\Users\Lermars asus nb\AppData\Local\Temp\ipykernel_11176\1420843906.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df6_3 = pd.read_sql_query(query6_3, connection)


,count
0,201



#### узнаем насколько популярен Python в требованиях работодателей к DS. Вычесляем количество вакансий, в которых в качестве ключевого навыка указан Python.

In [23]:
query6_4= f'''select COUNT(*)
              from vacancies
              where
                    (
                    lower(name) like '%data scientist%'  or
                    lower(name) like '%data science%' or
                    lower(name) like '%исследователь данных%'  or
                    (name like '%ML%' and name not ilike '%HTML%')  or
                    lower(name) like '%machine learning%'  or
                    lower(name) like '%машинн%обучен%'
                    )
                    and
                    (
                    key_skills ilike '%python%' 
                )
           '''
df6_4 = pd.read_sql_query(query6_4, connection)
df6_4

C:\Users\Lermars asus nb\AppData\Local\Temp\ipykernel_11176\3404789317.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df6_4 = pd.read_sql_query(query6_4, connection)


,count
0,351


#### Узнаем Сколько ключевых навыков в среднем указывают в вакансиях для DS

In [24]:
query6_5 = f'''select AVG(length(key_skills) - length(replace(key_skills, CHR(9), '')) + 1) 
               from vacancies
               where
                    (
                    lower(name) like '%data scientist%'  or
                    lower(name) like '%data science%' or
                    lower(name) like '%исследователь данных%'  or
                    (name like '%ML%' and name not ilike '%HTML%')  or
                    lower(name) like '%machine learning%'  or
                    lower(name) like '%машинн%обучен%'
                    )
                    and
                    (
                    (key_skills) IS NOT NULL 
                    )
            '''
df6_5 = pd.read_sql_query(query6_5,connection)
df6_5
#Тагже это можно вечеслить с помощью библиотеки pandas если заменить первую строку запроса на select * 
#df6_5.dropna(inplace=True)
#df6_5['key_skills'] = df6_5['key_skills'].apply(lambda x: x.split(sep='\t'))
#result12= df6_5['key_skills'].apply(lambda x: len(x))
#result12.mean()

C:\Users\Lermars asus nb\AppData\Local\Temp\ipykernel_11176\3480064133.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df6_5 = pd.read_sql_query(query6_5,connection)


,avg
0,6.406032


#### Узнаем какую зарплату для DS в среднем указывают для каждого типа требуемого опыта (уникальное значение из поля experience)

In [25]:
query6_6= f'''SELECT experience, round(avg(coalesce((salary_from+salary_to)/2, salary_from, salary_to)),0)  as avg_DS_salary 
              FROM vacancies 
              WHERE (name ilike '%data science%'
                    or name ilike '%data scientist%'
                    or name ilike '%machine learning%'
                    or name ilike '%исследователь данных%'
                    or name ilike '%машинн%обучен%'
                    or (name like '%ML%' and name not like '%HTML%'))
                    
              GROUP BY 1 
           '''
df6_6=pd.read_sql_query(query6_6,connection)
df6_6

C:\Users\Lermars asus nb\AppData\Local\Temp\ipykernel_11176\3370446287.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df6_6=pd.read_sql_query(query6_6,connection)


,experience,avg_ds_salary
0,Более 6 лет,NaN
1,Нет опыта,74643.0
2,От 1 года до 3 лет,139675.0
3,От 3 до 6 лет,243115.0


### Из интерестно можно увидеть что для опыта свыше 6 лет зарплата имеет значение "NaN" что может указывать на то что она не указывается и зависит скорее всего от собеседований и желаний / возможностей обеих старон . В связи с высоким опытом (и уровнем как следсвие) соискателя . 

In [26]:
connection.close()